In [2]:
x = 1

In [1]:
import matplotlib.pyplot as plt

import jax
import jax.random as jrnd
import jax.numpy as jnp
import distrax as dx
import jaxkern as jk

from jax.config import config
config.update("jax_enable_x64", True)  # crucial for Gaussian processes

from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

from uicsmodels.gaussianprocesses.fullgp import FullLatentGPModel, FullMarginalGPModel

<ipython-input-1-7528ff77912c>:9: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config


In [24]:
priors = dict(kernel=dict(lengthscale=dx.Transformed(dx.Normal(loc=0.,
                                                               scale=1.),
                                                     tfb.Exp()),
                          variance=dx.Transformed(dx.Normal(loc=0.,
                                                            scale=1.),
                                                  tfb.Exp()),
                          num=dx.Normal(loc=0.,
                                        scale=1.)
                                       ),
              likelihood=dict(obs_noise=dx.Transformed(dx.Normal(loc=0.,
                                                                 scale=1.),
                                                       tfb.Exp())),
              hyper = dict(hyper_pp = dx.Transformed(dx.Normal(loc=0.,
                                                               scale=1.),
                                                     tfb.Exp())))

In [10]:
from jax.tree_util import tree_flatten, tree_unflatten
from distrax._src.distributions.distribution import Distribution
from distrax._src.bijectors.bijector import Bijector

priors_flat, _ = tree_flatten(priors, lambda l: isinstance(l, (Distribution, Bijector)))
priors_flat

In [25]:
orig_names = [*priors]
names = []
for name in orig_names:
  names = names + [*priors[name]]

print(names)

['lengthscale', 'variance', 'num', 'obs_noise', 'hyper_pp']


In [21]:
jnp.where(names == 'variance')

(Array([], dtype=int64),)

In [19]:
logprob = 0
for dist, name in zip(priors_flat, names):
  if name == 'num':
    logprob += jnp.sum(dist.log_prob(name, names[-1]))

test
